In [0]:
CREATE OR REPLACE TABLE sales_example (
  sale_id INT,
  customer_id INT,
  product_id INT,
  sale_date DATE,
  amount DECIMAL(10,2)
);

INSERT INTO sales_example VALUES
  (1, 101, 1001, DATE '2026-01-20', 250.00),
  (2, 102, 1002, DATE '2026-01-21', 175.50),
  (3, 103, 1003, DATE '2026-01-21', 320.75),
  (4, 104, 1004, DATE '2026-01-22', 89.99),
  (5, 105, 1005, DATE '2026-01-23', 150.00);

In [0]:
-- REFRESH POLICY refresh_policy

-- refresh_policy:
--   AUTO | INCREMENTAL | INCREMENTAL STRICT | FULL

-- The system uses incremental refreshes when possible.
-- STRICT: The system uses incremental refreshes.

In [0]:
CREATE OR REPLACE MATERIALIZED VIEW sales_example_agg
  REFRESH POLICY INCREMENTAL STRICT AS
SELECT
  sale_date,
  SUM(amount) AS total_amount
FROM
  sales_example
GROUP BY
  sale_date;

In [0]:
UPDATE sales_example SET amount = 1000.00 WHERE sale_id = 1;

In [0]:
REFRESH MATERIALIZED VIEW sales_example_agg

In [0]:
CREATE OR REPLACE TABLE customers_example (
  customer_id INT,
  segment STRING,
  valid_from DATE,
  valid_to DATE
);

INSERT INTO customers_example VALUES
  (101, 'Retail', DATE '2026-01-01', DATE '2026-01-31'),
  (101, 'VIP',    DATE '2026-01-15', DATE '2026-02-15'),
  (102, 'Retail', DATE '2026-01-01', DATE '2026-12-31'),
  (103, 'SMB',    DATE '2026-01-01', DATE '2026-12-31'),
  (104, 'Retail', DATE '2026-01-01', DATE '2026-12-31'),
  (105, 'VIP',    DATE '2026-01-01', DATE '2026-12-31');


In [0]:
CREATE OR REPLACE TABLE product_price_history (
  product_id INT,
  price DECIMAL(10,2),
  valid_from DATE,
  valid_to DATE
);

INSERT INTO product_price_history VALUES
  (1001, 200.00, DATE '2026-01-01', DATE '2026-01-31'),
  (1001, 220.00, DATE '2026-01-15', DATE '2026-02-15'), -- overlap
  (1002, 175.50, DATE '2026-01-01', DATE '2026-12-31'),
  (1003, 320.75, DATE '2026-01-01', DATE '2026-12-31'),
  (1004,  89.99, DATE '2026-01-01', DATE '2026-12-31'),
  (1005, 150.00, DATE '2026-01-01', DATE '2026-12-31');


In [0]:
CREATE OR REPLACE TABLE fx_rates (
  rate_date DATE,
  currency STRING,
  rate DECIMAL(10,4)
);

INSERT INTO fx_rates VALUES
  (DATE '2026-01-20', 'EUR', 1.10),
  (DATE '2026-01-21', 'EUR', 1.11),
  (DATE '2026-01-22', 'EUR', 1.12),
  (DATE '2026-01-23', 'EUR', 1.10);


In [0]:
CREATE OR REPLACE MATERIALIZED VIEW sales_example_agg_joins
REFRESH POLICY INCREMENTAL STRICT
AS
SELECT
  s.sale_date,
  c.segment,
  SUM(s.amount * fx.rate) AS total_amount_eur
FROM sales_example s
JOIN customers_example c
  ON s.customer_id = c.customer_id
 AND s.sale_date BETWEEN c.valid_from AND c.valid_to
JOIN product_price_history p
  ON s.product_id = p.product_id
 AND s.sale_date BETWEEN p.valid_from AND p.valid_to
JOIN (
  SELECT rate_date, MAX(rate) AS rate
  FROM fx_rates
  WHERE currency = 'EUR'
  GROUP BY rate_date
) fx
  ON fx.rate_date = s.sale_date
GROUP BY
  s.sale_date,
  c.segment;
